In [35]:
# carregamento de bibliotecas
import os
import sys
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
# Adiciona a pasta raiz do projeto (onde está a pasta src) ao sys.path
sys.path.append(os.path.abspath(os.path.join("..")))
from src.config.logging_config import setup_logging
from src.model.train import *


In [36]:
# configuração do logging
setup_logging()

In [37]:
# carregamento do dataset processado
tbl_cotacao_ibovespa = pd.read_csv('../data/processed/tbl_cotacao_ibovespa_processed.csv',index_col=0, parse_dates=True)
tbl_cotacao_ibovespa.index = tbl_cotacao_ibovespa.index.tz_localize(None)


# Seleção de features

In [38]:
#filtro = ~tbl_cotacao_ibovespa.columns.str.startswith(('retorno',))
tbl_cotacao_ibovespa = tbl_cotacao_ibovespa.drop(columns=['fechamento', 'month','dayofweek','abertura','feriado','tipo'])
#tbl_cotacao_ibovespa = tbl_cotacao_ibovespa[tbl_cotacao_ibovespa.columns[filtro]]

In [39]:
tbl_cotacao_ibovespa.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12117 entries, 2024-12-16 to 2025-07-24
Data columns (total 73 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   acao                             12117 non-null  object 
 1   maximo                           12117 non-null  float64
 2   minimo                           12117 non-null  float64
 3   volume                           12117 non-null  int64  
 4   setor                            12117 non-null  object 
 5   industria                        12117 non-null  object 
 6   day_sin                          12117 non-null  float64
 7   day_cos                          12117 non-null  float64
 8   month_sin                        12117 non-null  float64
 9   month_cos                        12117 non-null  float64
 10  close_diff                       12117 non-null  float64
 11  lag_1_close_diff                 12117 non-null  float64
 12  l

In [40]:
# visualização dos dados selecionados
tbl_cotacao_ibovespa.head()

,acao,maximo,minimo,volume,setor,industria,day_sin,day_cos,month_sin,month_cos,...,rolling_mean_60_Volume,volatility_60_Volume,retorno_acumulado_60_Volume,rolling_mean_90_Volume,volatility_90_Volume,retorno_acumulado_90_Volume,rsi,macd,macd_signal,retorno
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-16,ABEV3,12.488062,11.948339,32095500,Consumo não-cíclico,Bebidas,0.000000,1.000000,-2.449294e-16,1.0,...,3.329825e+07,1.511081e+07,1.997895e+09,3.077329e+07,1.381457e+07,2.769596e+09,53.080443,0.012518,0.016665,-0.006633
2024-12-17,ABEV3,12.274033,12.087922,39564500,Consumo não-cíclico,Bebidas,0.951057,0.309017,-2.449294e-16,1.0,...,3.308795e+07,1.503688e+07,1.985277e+09,3.081687e+07,1.381246e+07,2.773519e+09,56.951273,0.010027,0.015338,0.007229
2024-12-18,ABEV3,12.171673,11.864590,40864200,Consumo não-cíclico,Bebidas,0.587785,-0.809017,-2.449294e-16,1.0,...,3.305716e+07,1.502142e+07,1.983429e+09,3.104401e+07,1.378604e+07,2.793961e+09,54.425876,0.006025,0.013475,-0.009412
2024-12-19,ABEV3,12.013478,11.762228,38104000,Consumo não-cíclico,Bebidas,-0.587785,-0.809017,-2.449294e-16,1.0,...,3.340474e+07,1.495553e+07,2.004284e+09,3.130360e+07,1.374808e+07,2.817324e+09,50.657092,0.002936,0.011367,0.000565
2024-12-20,ABEV3,12.076701,11.791965,86074600,Consumo não-cíclico,Bebidas,-0.951057,0.309017,-2.449294e-16,1.0,...,3.366316e+07,1.489950e+07,2.019790e+09,3.139273e+07,1.376606e+07,2.825346e+09,45.052726,0.001194,0.009333,0.003487


# Separação dos dados

In [41]:
# separação dos dados em treino e teste
X_treino, X_teste,y_treino,y_teste = split_data_by_date(df=tbl_cotacao_ibovespa,cutoff_date='2025-06-24',
                                                        target='close_diff')

Dados de treino: 2024-12-16 a 2025-06-23
Dados de teste: 2025-06-24 a 2025-07-24


# Rodada de Seleção de Modelo: Sarimax

In [42]:
# Junta os dataframes de treino
dataset_treino = pd.concat([X_treino, y_treino], axis=1)

# Remove ação problemática (poucos dados após pré-processamento)
dataset_treino = dataset_treino[dataset_treino['acao'] != 'TIMS3']

# Remove colunas não relevantes
dataset_treino.drop(columns=['setor', 'industria'], axis=1, inplace=True)

In [ ]:
# treino do modelo do modelo SARIMAX
coluna_target = 'close_diff'

feat_exog = [
    'day_sin','day_cos','month_sin','month_cos',
    'volume','rsi','lag_1_Volume','lag_3_Volume','lag_5_Volume','lag_7_Volume','lag_15_Volume',
    'lag_30_Volume','lag_60_Volume','lag_90_Volume','rolling_mean_3_Volume','volatility_3_Volume',
    'rolling_mean_5_Volume','volatility_5_Volume','rolling_mean_7_Volume','volatility_7_Volume',
    'rolling_mean_15_Volume','volatility_15_Volume','rolling_mean_30_Volume','volatility_30_Volume',
    'rolling_mean_60_Volume','volatility_60_Volume','rolling_mean_90_Volume','volatility_90_Volume','macd','macd_signal'
]

for acao in X_treino.

2025-08-08 18:36:47,859 | INFO | src.model.train | Usando parâmetros ARIMA: (1, 0, 1). Sazonalidade removida.
2025-08-08 18:36:47,862 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: ABEV3
2025-08-08 18:36:48,008 | INFO | src.model.train | Modelo SARIMAX para ABEV3 treinado com sucesso.
2025-08-08 18:36:48,010 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: PETR4


c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\t

2025-08-08 18:36:48,152 | INFO | src.model.train | Modelo SARIMAX para PETR4 treinado com sucesso.
2025-08-08 18:36:48,153 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: UGPA3
2025-08-08 18:36:48,290 | INFO | src.model.train | Modelo SARIMAX para UGPA3 treinado com sucesso.
2025-08-08 18:36:48,291 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: GOAU4


c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\base\model

2025-08-08 18:36:48,430 | INFO | src.model.train | Modelo SARIMAX para GOAU4 treinado com sucesso.
2025-08-08 18:36:48,431 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: COGN3
2025-08-08 18:36:48,573 | INFO | src.model.train | Modelo SARIMAX para COGN3 treinado com sucesso.
2025-08-08 18:36:48,575 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: CSNA3


...

2025-08-08 18:36:54,014 | INFO | src.model.train | Modelo SARIMAX para SMFT3 treinado com sucesso.
2025-08-08 18:36:54,014 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: EMBR3
2025-08-08 18:36:54,151 | INFO | src.model.train | Modelo SARIMAX para EMBR3 treinado com sucesso.
2025-08-08 18:36:54,152 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: ELET6


c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\base\model

2025-08-08 18:36:54,290 | INFO | src.model.train | Modelo SARIMAX para ELET6 treinado com sucesso.
2025-08-08 18:36:54,291 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: SUZB3
2025-08-08 18:36:54,428 | INFO | src.model.train | Modelo SARIMAX para SUZB3 treinado com sucesso.
2025-08-08 18:36:54,429 | INFO | src.model.train | Treinando modelo SARIMAX para a ação: AURE3


c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\statsmodels\base\model

KeyboardInterrupt: 

In [18]:
# concatenação dos dados de teste para validação de resultado
dataset_teste = pd.concat([X_teste, y_teste], axis=1).drop(columns=['setor', 'industria'], axis=1)

Processo de seleção de Modelo

In [496]:
# pré-processamento dos dados: target encoding, onehot encoding e transformação e scaler. 

# separação das colunas categoricas 
cols_cat_oh = ['setor','industria']
cols_nums = tbl_cotacao_ibovespa.drop(columns=['close_diff'],axis=1).select_dtypes('number').columns.to_list()

# target encoder inserido na função
preprocessor = criar_preprocessor(target_encoder='acao', colunas_categoricas_onehot=cols_cat_oh, colunas_numericas=cols_nums)
preprocessor

2025-08-08 15:00:30,220 | INFO | src.model.train | Iniciando construção do pipeline de pré-processamento.
2025-08-08 15:00:30,221 | INFO | src.model.train | Pipeline de pré-processamento construído com sucesso.


,transformers,"[('target_encoder', ...), ('cat_features', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,verbose,0
,cols,['acao']
,drop_invariant,False


In [448]:
preprocessor = criar_preprocessor_sem_target_encoder(
    colunas_categoricas_onehot=teste.select_dtypes('object').columns.to_list(),
    colunas_numericas=tbl_cotacao_ibovespa.drop(columns='close_diff',axis=1).select_dtypes('number').columns.to_list()
                                                     )

2025-08-08 03:24:21,856 | INFO | src.model.train | Iniciando construção do pipeline de pré-processamento.
2025-08-08 03:24:21,856 | INFO | src.model.train | Pipeline de pré-processamento construído com sucesso.


XGBOOST

In [497]:
# construção do pipeline para o modelo XGboost

model_xgb = criar_pipeline(preprocessor=preprocessor, modelo_final=XGBRegressor(random_state=42))
model_xgb

2025-08-08 15:00:37,280 | INFO | src.model.train | Iniciando criação do pipeline completo.
2025-08-08 15:00:37,295 | INFO | src.model.train | ColumnTransformer(transformers=[('target_encoder', TargetEncoder(cols=['acao']),
                                 ['acao']),
                                ('cat_features',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['setor', 'industria']),
                                ('num_features',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('power_transf...
                                

,steps,"[('preprocessador', ...), ('modelo', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('target_encoder', ...), ('cat_features', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [498]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

ts = TimeSeriesSplit(n_splits=5)
param_grid_xgb = {
    'modelo__n_estimators': [200, 500],
    'modelo__max_depth': [3, 6, 9],
    'modelo__learning_rate': [0.01, 0.1],
    'modelo__subsample': [0.8, 1.0],
    'modelo__colsample_bytree': [0.8, 1.0]
}

random = RandomizedSearchCV(estimator=model_xgb,param_distributions=param_grid_xgb,cv=ts, n_iter=100, 
                            random_state=42, n_jobs=-1, verbose=5)

In [499]:
random.fit(X_treino,y_treino)
y_pred_xgb = random.predict(X_teste)

c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [500]:
# avaliação de resultado
gerar_metricas(y_pred=y_pred_xgb, y_true=y_teste)

MAE: 0.0101
RMSE: 0.0137
R2 Score: 0.4013


Random Forest

In [501]:
# construção do pipeline para o modelo Random Forest Regressor
model_rf = criar_pipeline(preprocessor=preprocessor, modelo_final=RandomForestRegressor())


2025-08-08 15:10:48,914 | INFO | src.model.train | Iniciando criação do pipeline completo.
2025-08-08 15:10:48,925 | INFO | src.model.train | ColumnTransformer(transformers=[('target_encoder', TargetEncoder(cols=['acao']),
                                 ['acao']),
                                ('cat_features',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['setor', 'industria']),
                                ('num_features',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('power_transf...
                                

In [502]:
from sklearn.model_selection import TimeSeriesSplit

ts = TimeSeriesSplit(n_splits=5)

param_grid = {
    'modelo__n_estimators': [100, 200, 500],
    'modelo__max_depth': [5, 10, 20, 30, None],
    'modelo__min_samples_split': [2, 5, 10, 15],
    'modelo__min_samples_leaf': [1, 2, 4, 8],
    'modelo__max_features': ['sqrt', 'log2', 0.8, 1.0]
}

random = RandomizedSearchCV(estimator=model_xgb,
                          param_distributions=param_grid_xgb,cv=ts, 
                          n_iter=100, random_state=42, n_jobs=-1, verbose=5)


In [503]:
random.fit(X_treino, y_treino)
y_pred_rf = random.predict(X_teste)

c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [504]:
# avaliação de resultado
gerar_metricas(y_pred=y_pred_rf, y_true=y_teste)

MAE: 0.0101
RMSE: 0.0137
R2 Score: 0.4013


CATBOOST

In [505]:
# construção do pipeline para o modelo Catboost
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
colunas_numericas = X_treino.select_dtypes(include=np.number).columns.tolist()
colunas_categoricas = ['acao','setor']
todas_colunas = colunas_numericas + colunas_categoricas

ts = TimeSeriesSplit(n_splits=5)
# Verifique se as listas de colunas estão corretas
print(f"Colunas numéricas: {colunas_numericas}")
print(f"Colunas categóricas: {colunas_categoricas}")

# 2. Crie o pré-processador
preprocessor_catboost = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('norm', PowerTransformer(method='yeo-johnson')),
            ('scaler', StandardScaler())
        ]), colunas_numericas),
        ('cat', 'passthrough', colunas_categoricas)
    ],
    remainder='drop' # 'drop' para garantir que apenas as colunas que você especificou sejam usadas
)

# 3. Mapeie os índices das colunas categóricas APÓS a transformação
# O ColumnTransformer irá colocar as colunas numéricas primeiro, seguidas pelas categóricas
cat_features_indices = list(range(len(colunas_numericas), len(colunas_numericas) + len(colunas_categoricas)))
print(f"Índices das colunas categóricas para o CatBoost: {cat_features_indices}")

# 4. Instancie o modelo CatBoost
catboost_model = CatBoostRegressor(
    random_state=42, 
    verbose=0,
    cat_features=cat_features_indices,
    # É importante definir aqui, não na busca de parâmetros
)

# 5. Crie o pipeline final
model_cb = Pipeline([
    ('preprocessor', preprocessor_catboost),
    ('model', catboost_model)
])

# 6. Dicionário de parâmetros para o RandomizedSearchCV
# Note que 'model__cat_features' foi removido para evitar a duplicação
param_grid_catboost = {
    'model__iterations': [100, 250, 500, 750, 1000],
    'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'model__depth': [4, 6, 8, 10],
    'model__l2_leaf_reg': [1, 3, 5, 7, 9],
    'model__subsample': [0.6, 0.8, 1.0],
    'model__border_count': [32, 64, 128],
    'model__verbose': [0]
}

# 7. Crie o objeto RandomizedSearchCV
random = RandomizedSearchCV(
    estimator=model_cb,
    param_distributions=param_grid_catboost,
    cv=ts, 
    n_iter=100, 
    random_state=42, 
    n_jobs=-1
)

Colunas numéricas: ['volume', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'lag_1_close_diff', 'lag_3_close_diff', 'lag_5_close_diff', 'lag_7_Close_diff', 'lag_15_Close_diff', 'lag_30_Close_diff', 'lag_60_Close_diff', 'lag_90_Close_diff', 'rolling_mean_3_close_diff', 'volatility_3_close_diff', 'rolling_mean_5_close_diff', 'volatility_5_close_diff', 'rolling_mean_7_Close_diff', 'volatility_7_Close_diff', 'rolling_mean_15_Close_diff', 'volatility_15_Close_diff', 'rolling_mean_30_Close_diff', 'volatility_30_Close_diff', 'rolling_mean_60_Close_diff', 'volatility_60_Close_diff', 'rolling_mean_90_Close_diff', 'volatility_90_Close_diff', 'lag_1_Volume', 'lag_3_Volume', 'lag_5_Volume', 'lag_7_Volume', 'lag_15_Volume', 'lag_30_Volume', 'lag_60_Volume', 'lag_90_Volume', 'rolling_mean_3_Volume', 'volatility_3_Volume', 'rolling_mean_5_Volume', 'volatility_5_Volume', 'rolling_mean_7_Volume', 'volatility_7_Volume', 'rolling_mean_15_Volume', 'volatility_15_Volume', 'rolling_mean_30_Volume', 'volat

In [506]:
# ajuste e previsão do modelo
random.fit(X_treino, y_treino)
y_pred_cb = random.predict(X_teste)

KeyboardInterrupt: 

In [238]:
# avaliação de resultado
gerar_metricas(y_pred=y_pred_cb, y_true=y_teste)

MAE: 0.0140
RMSE: 0.0190
R2 Score: -0.1911
